In [1]:
import os
from collections import defaultdict
vocabulary = defaultdict(list)
category_frequency = defaultdict(float)
total_documents = 0
test_df = defaultdict(list)



def classify():
    global total_documents

    for directory in os.listdir('dataset'):        
        ctr = 0
        if os.path.isdir(f'dataset/{directory}'):
        
            category = directory                        
            
            length = len(os.listdir(f'dataset/{category}'))
            
            
            for filename in os.listdir(f'dataset/{category}'):
                
                # Check if the file is a regular file (not a directory)
                if os.path.isfile(os.path.join(f'dataset/{category}', filename)):
                    
                    with open(os.path.join(f'dataset/{category}', filename), 'r') as file:
                        
                        words = file.read().split()
                        
                        if ctr < 0.8*(length):
                            for word in words:
                                vocabulary[category].append(word)
                        else:
                            test_df[category].append(words)

                        ctr += 1

                category_frequency[category] += 1
                total_documents += 1 

In [2]:
classify()
vocabulary

defaultdict(list,
            {'business': ['Ad',
              'sales',
              'boost',
              'Time',
              'Warner',
              'profit',
              'Quarterly',
              'profits',
              'at',
              'US',
              'media',
              'giant',
              'TimeWarner',
              'jumped',
              '76%',
              'to',
              '$1.13bn',
              '(Â£600m)',
              'for',
              'the',
              'three',
              'months',
              'to',
              'December,',
              'from',
              '$639m',
              'year-earlier.',
              'The',
              'firm,',
              'which',
              'is',
              'now',
              'one',
              'of',
              'the',
              'biggest',
              'investors',
              'in',
              'Google,',
              'benefited',
              'from',
              'sales',
 

In [3]:
category_frequency

defaultdict(float,
            {'business': 510.0,
             'entertainment': 386.0,
             'politics': 417.0,
             'sport': 511.0,
             'tech': 401.0})

In [4]:
from math import log
logiprior = defaultdict(float)
for category in category_frequency:
    logiprior[category] = log(category_frequency[category] / total_documents)

logiprior

defaultdict(float,
            {'business': -1.4731014688819692,
             'entertainment': -1.7516748251355099,
             'politics': -1.6744259728015392,
             'sport': -1.471142604396636,
             'tech': -1.7135507672937715})

In [5]:
word_count = defaultdict(int)
words_vocabulary = set()

for category in vocabulary:
    for word in vocabulary[category]:
        word_count[word] += 1
        words_vocabulary.add(word)

word_count

defaultdict(int,
            {'Ad': 2,
             'sales': 316,
             'boost': 115,
             'Time': 12,
             'Warner': 21,
             'profit': 69,
             'Quarterly': 2,
             'profits': 107,
             'at': 3537,
             'US': 1057,
             'media': 169,
             'giant': 118,
             'TimeWarner': 7,
             'jumped': 29,
             '76%': 2,
             'to': 19620,
             '$1.13bn': 2,
             '(Â£600m)': 1,
             'for': 6821,
             'the': 35220,
             'three': 535,
             'months': 239,
             'December,': 52,
             'from': 2783,
             '$639m': 1,
             'year-earlier.': 1,
             'The': 5732,
             'firm,': 36,
             'which': 2061,
             'is': 6751,
             'now': 717,
             'one': 1076,
             'of': 15698,
             'biggest': 212,
             'investors': 65,
             'in': 13065,
             'G

In [6]:
from collections import Counter
loglikelihood = defaultdict(int)

for category in vocabulary:
    counter = Counter(vocabulary[category])
    for word in words_vocabulary:
        loglikelihood[(word, category)] = log((counter[word] + 1) / (len(vocabulary[category]) + 1))

loglikelihood

defaultdict(int,
            {('con', 'business'): -11.813844540454005,
             ('task', 'business'): -11.12069735989406,
             ('adaptations', 'business'): -11.813844540454005,
             ('thoughts,', 'business'): -11.813844540454005,
             ('cavalier', 'business'): -11.813844540454005,
             ('downward', 'business'): -10.427550179334114,
             ('34', 'business'): -10.715232251785896,
             ('meetings', 'business'): -10.427550179334114,
             ('Warwick.', 'business'): -11.813844540454005,
             ('healing."', 'business'): -11.813844540454005,
             ('films".', 'business'): -11.813844540454005,
             ('congestive', 'business'): -11.813844540454005,
             ('enjoy.', 'business'): -11.813844540454005,
             ('assassinated', 'business'): -11.12069735989406,
             ('basic', 'business'): -10.02208507122595,
             ('Easymobile', 'business'): -10.02208507122595,
             ("Cent's", 'business')

In [7]:
def test(test_df, logiprior, loglikelihood, categories, words_vocabulary):
    ans = defaultdict(int)
    
    for category in categories:
        summ = logiprior[category]

        for word in test_df:
            if word in words_vocabulary:
                summ += loglikelihood[(word, category)]


        ans[category] = summ


    output = -float('inf')
    for category in ans:
        if ans[category] > output:
            best = category
            output = ans[category]

    return best

In [8]:
categories = []
for cat in vocabulary:
    categories.append(cat)

In [9]:
ctr = total = 0
for category in test_df:
    for doc in test_df[category]:
        output = test(doc, logiprior, loglikelihood, categories, words_vocabulary)
        if output == category:
            ctr += 1
        total += 1


print(f'Acurracy: {ctr / total * 100}%')

Acurracy: 99.09909909909909%
